# Crear _wrapper_

Creamos un _wrapper_ que modifica las salidas de un modelo **ya entrenado** para que sea compatible con el formato `numpy`:

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin

class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        # Asume que el modelo ya está entrenado, por lo que no hacemos nada en fit
        return self

    def predict(self, X):
        return self.model.predict(X).flatten()

# Cargar modelo

Cargamos el modelo ya entrenado y le aplicamos el _wrapper_:

In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model_tf = load_model("model.keras")
model = KerasRegressorWrapper(model_tf)

2024-11-11 11:15:46.976662: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-11 11:15:46.987365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731320146.999210   82345 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731320147.002624   82345 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-11 11:15:47.015298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Cargar datos

Cargamos los datos de **calibración** y *_test_*:

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing()
X, y = data.data, data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_cal, X_test, y_cal, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Cálculo de incertidumbre

Utilizamos `MapieRegressor`para calcular los intervalos de las predicciones:

In [5]:
from mapie.regression import MapieRegressor

mapie_regressor = MapieRegressor(estimator=model, method="plus")

mapie_regressor.fit(X_cal, y_cal)
y_preds, y_intervals = mapie_regressor.predict(X_test, alpha=0.1)

I0000 00:00:1731320149.106033   82448 service.cc:148] XLA service 0x7a4df0004b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731320149.106052   82448 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-11-11 11:15:49.109562: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1731320149.118485   82448 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-11 11:15:49.131679: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


  1/104 ━━━━━━━━━━━━━━━━━━━━ 33s 328ms/step

I0000 00:00:1731320149.388457   82448 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step 
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 427us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 426us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step


Vemos algunos ejemplos:

In [17]:
for i in range(5):
    print(f"Predicción: {y_preds[i]:.2f} | Intervalo: {y_intervals[i].flatten()}")


Predicción: 0.39 | Intervalo: [-0.35770522  1.12862848]
Predicción: 0.92 | Intervalo: [0.17989995 1.66623364]
Predicción: 4.87 | Intervalo: [4.12870701 5.6150407 ]
Predicción: 2.73 | Intervalo: [1.98930962 3.47564332]
Predicción: 2.68 | Intervalo: [1.93476422 3.42109792]
